# Analise de dados imputados    

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.interpolate import Akima1DInterpolator
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Carregando e preparando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'],
                          parse_dates=['Data e Hora'],
                          index_col='Data e Hora',
                          low_memory=False)

df_original.sort_index(inplace=True)

# Selecionando todas as colunas e focando em um período específico
df = df_original.loc['2019-01-01':'2021-01-31']
df = df.apply(pd.to_numeric, errors='coerce')
df = df.asfreq('h')

# Encontrar a sequência mais longa de dados não ausentes para PM2.5
mask = df['PM2.5'].notna()
id_groups = mask.ne(mask.shift()).cumsum()
longest_sequence = df[mask].groupby(id_groups).filter(lambda x: len(x) == mask.groupby(id_groups).size().max())

print(f"Tamanho da sequência mais longa: {len(longest_sequence)}")
print(f"Número de valores não nulos na sequência mais longa: {longest_sequence['PM2.5'].notna().sum()}")

if len(longest_sequence) == 0:
    raise ValueError("A sequência mais longa está vazia. Verifique seus dados.")

# Métodos de imputação
methods = {
    'Original': None,
    'MICE': IterativeImputer(random_state=0, max_iter=1000),
    'Akima Interpolation': lambda x: pd.Series(Akima1DInterpolator(x.dropna().index.astype(int), x.dropna().values)(x.index.astype(int)), index=x.index),
    # 'Forward Fill': lambda x: x.fillna(method='ffill'),
    'Backward Fill': lambda x: x.fillna(method='bfill'),
    'Linear Interpolation': lambda x: x.interpolate(method='linear'),
    # 'KNN': KNNImputer(n_neighbors=5),
    # 'Mean': SimpleImputer(strategy='mean'),
    # 'Exponential Smoothing': lambda x: ExponentialSmoothing(x.fillna(x.mean())).fit().fittedvalues,
    'Random Forest': IterativeImputer(estimator=RandomForestRegressor(n_estimators=1000), random_state=0, max_iter=100),
    'XGBoost': IterativeImputer(estimator=XGBRegressor(n_estimators=1000), random_state=0, max_iter=100),
}

# Função para avaliar os métodos de imputação
def evaluate_imputation(original, imputed):
    mask = ~np.isnan(original) & ~np.isnan(imputed)
    original = original[mask]
    imputed = imputed[mask]
    
    if len(original) == 0 or len(imputed) == 0:
        print("Aviso: Não há dados válidos para comparação após a remoção de NaN.")
        return np.nan, np.nan
    
    mse = mean_squared_error(original, imputed)
    mae = mean_absolute_error(original, imputed)
    return mse, mae

# Criar dados artificiais com valores ausentes na sequência mais longa
test_data = longest_sequence.copy()
np.random.seed(0)
mask = np.random.rand(len(test_data)) < 0.1
test_data.loc[mask, 'PM2.5'] = np.nan

print(f"Porcentagem de valores ausentes nos dados de teste: {test_data['PM2.5'].isna().mean()*100:.2f}%")

# Avaliar cada método
results = {}
imputed_data = {}
for method_name, method in methods.items():
    if method_name == 'Original':
        continue
    
    try:
        if isinstance(method, SimpleImputer) or isinstance(method, IterativeImputer) or isinstance(method, KNNImputer):
            imputed = pd.DataFrame(method.fit_transform(test_data), index=test_data.index, columns=test_data.columns)
        else:
            imputed = test_data.copy()
            for col in test_data.columns:
                imputed[col] = method(test_data[col])
        
        mse, mae = evaluate_imputation(longest_sequence['PM2.5'], imputed['PM2.5'])
        results[method_name] = {'MSE': mse, 'MAE': mae}
        imputed_data[method_name] = imputed
    except Exception as e:
        print(f"Erro ao avaliar o método {method_name}: {str(e)}")
        results[method_name] = {'MSE': np.nan, 'MAE': np.nan}

# Ordenar os resultados pelo MSE
results_df = pd.DataFrame(results).T.sort_values('MSE')

print("Resultados ordenados pelo MSE:")
print(results_df)

# Plotar os resultados de MSE e MAE
if not results_df.empty and not results_df['MSE'].isnull().all():
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Mean Squared Error', 'Mean Absolute Error'))

    fig.add_trace(go.Bar(x=results_df.index, y=results_df['MSE'], name='MSE'), row=1, col=1)
    fig.add_trace(go.Bar(x=results_df.index, y=results_df['MAE'], name='MAE'), row=1, col=2)

    fig.update_layout(title='Comparação dos Métodos de Imputação', showlegend=False)
    fig.show()

# Plotar a série temporal original, com dados faltantes e imputados
fig = make_subplots(rows=1, cols=1, subplot_titles=('Comparação das Séries Temporais Imputadas'))

# Adicionar a série temporal original
fig.add_trace(go.Scatter(x=longest_sequence.index, y=longest_sequence['PM2.5'], mode='lines', name='Original'))

# Adicionar a série com dados faltantes
fig.add_trace(go.Scatter(x=test_data.index, y=test_data['PM2.5'], mode='lines', name='Dados Faltantes', line=dict(dash='dash')))

# Adicionar as séries imputadas
for method_name, imputed in imputed_data.items():
    fig.add_trace(go.Scatter(x=imputed.index, y=imputed['PM2.5'], mode='lines', name=method_name))

fig.update_layout(title='Comparação das Séries Temporais Imputadas e Dados Faltantes',
                  xaxis_title='Data',
                  yaxis_title='PM2.5')
fig.show()

# Análise de importância de atributos
def feature_importance_analysis(data):
    # Preparar os dados
    X = data.drop('PM2.5', axis=1)
    y = data['PM2.5']
    
    # Treinar um modelo Random Forest
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X, y)
    
    # Calcular a importância dos atributos
    importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return importance

# Realizar análise de importância de atributos para cada método de imputação
for method_name, imputed in imputed_data.items():
    importance = feature_importance_analysis(imputed)
    print(f"\nImportância dos atributos para o método {method_name}:")
    print(importance)
    
    # Plotar a importância dos atributos
    fig = go.Figure(go.Bar(x=importance['feature'], y=importance['importance']))
    fig.update_layout(title=f'Importância dos Atributos - Método {method_name}',
                      xaxis_title='Atributos',
                      yaxis_title='Importância')
    fig.show()

# Análise de decomposição da série temporal
def plot_time_series_decomposition(data, title):
    decomposition = seasonal_decompose(data['PM2.5'], model='additive', period=24)
    
    fig = make_subplots(rows=4, cols=1, subplot_titles=('Observado', 'Tendência', 'Sazonal', 'Resíduo'))
    
    fig.add_trace(go.Scatter(x=data.index, y=decomposition.observed, mode='lines', name='Observado'), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=decomposition.trend, mode='lines', name='Tendência'), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=decomposition.seasonal, mode='lines', name='Sazonal'), row=3, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=decomposition.resid, mode='lines', name='Resíduo'), row=4, col=1)
    
    fig.update_layout(height=900, title_text=title)
    fig.show()

# Plotar decomposição para dados originais e métodos de imputação selecionados
plot_time_series_decomposition(longest_sequence, 'Decomposição da Série Temporal - Dados Originais')

for method_name in ['MICE', 'Random Forest', 'XGBoost']:
    if method_name in imputed_data:
        plot_time_series_decomposition(imputed_data[method_name], f'Decomposição da Série Temporal - Método {method_name}')

Tamanho da sequência mais longa: 397
Número de valores não nulos na sequência mais longa: 397
Porcentagem de valores ausentes nos dados de teste: 10.08%


C:\Users\portes\AppData\Local\Temp\ipykernel_15840\3397375138.py:43: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\portes\AppData\Local\Temp\ipykernel_15840\3397375138.py:44: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\dev\scripts\pm25-plots\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\dev\scripts\pm25-plots\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\dev\scripts\pm25-plots\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency h will be used.

C:\dev\scripts\pm25-plots\venv\lib\site-packages\

Resultados ordenados pelo MSE:
                             MSE       MAE
Random Forest           8.555225  0.523425
Linear Interpolation    8.918836  0.506297
Akima Interpolation     9.056030  0.512051
Backward Fill           9.652393  0.559194
MICE                    9.781671  0.527852
XGBoost                10.657200  0.550526
KNN                    10.787506  0.624685
Forward Fill           11.355164  0.498741
Mean                   16.164077  0.721440
Exponential Smoothing  71.546564  4.778376


C:\dev\scripts\pm25-plots\venv\lib\site-packages\sklearn\impute\_iterative.py:825: ConvergenceWarning:

[IterativeImputer] Early stopping criterion not reached.




Importância dos atributos para o método MICE:
               feature  importance
1  Monóxido de Carbono    0.569297
0                 PM10    0.430703


...


Importância dos atributos para o método KNN:
               feature  importance
1  Monóxido de Carbono    0.558425
0                 PM10    0.441575



Importância dos atributos para o método Mean:
               feature  importance
1  Monóxido de Carbono    0.528672
0                 PM10    0.471328



Importância dos atributos para o método Exponential Smoothing:
               feature  importance
0                 PM10    0.592607
1  Monóxido de Carbono    0.407393



Importância dos atributos para o método Random Forest:
               feature  importance
1  Monóxido de Carbono    0.575813
0                 PM10    0.424187



Importância dos atributos para o método XGBoost:
               feature  importance
1  Monóxido de Carbono     0.53416
0                 PM10     0.46584
